In [1]:
import json
from netCDF4 import Dataset
import matplotlib.pyplot as plt
#import matplotlib.animation as animation
import numpy as np
import simplesimdb as simplesim # this one is our own database manager
import subprocess # to capture errors from the simulations
import itertools
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2e' % x)

In [2]:
m = simplesim.Manager(directory='convergence', executable='./execute.sh', filetype='nc')
content = m.table()

In [3]:
###### THE RED BUTTON
#m.delete_all()

In [4]:
n2v={
    1:[144,1e-2],
    3:[48, 2e-3],
    4:[36, 2e-3],
    5:[24, 1e-3],
    7:[20, 5e-4]
}
endtime = 1
maxout = 5
reg2reg = {"none" : {"type" : "none"},
           "modal" : {"type" : "modal", "alpha" : 36, "order" : 8, "eta_c" : 0.5},
          "viscous": {"type" : "viscosity", "order" : 1,
                      "nu" : 1e-3, "direction" : "forward"},
          "hyperviscous" : {"type": "viscosity", "order" : 2,
                            "nu" : 5e-4, "direction" : "forward"}
          }

In [5]:
def generate_standard_input( n, multiple, multiplication, advection, regularization) :
    itstp=(endtime/maxout/n2v[n][1])*multiple
    if not itstp.is_integer() :
        print("Warning: itstp is not an integer!")
    inputfile={
        "grid":
        {   
            "type": "Cartesian2d",
            "n"  : n,
            "Nx" : int(multiple*n2v[n][0]),
            "Ny" : int(multiple*n2v[n][0]),
            "x": [-1.0, 1.0],
            "y": [-1.0, 1.0],
            "bc" : ["DIR", "PER"]
        },
        "timestepper":
        {   
            "type": "FilteredExplicitMultistep",
            "tableau": "eBDF-3-3",
            "dt" : n2v[n][1]/multiple
        },
        "regularization":  reg2reg[regularization],
        "output":
        {   
            "type": "netcdf",
            "itstp"   : int(itstp),
            "maxout"  : int(5)
        },
        "advection":
        {   
            "multiplication": multiplication,
            "type": advection
        },
        "elliptic":
        {   
            "type" : "multigrid",
            "stages": 3,
            "eps_pol" : [1e-6,10.0,10.0],
            "direction" : "forward"
        },
        "init":{ "type": "mms", "sigma": 0.2, "velocity": 1.0}
    }
    if regularization == "viscous" or regularization == "hyperviscous" :
        inputfile["timestepper"]["type"] = "ImExMultistep"
        inputfile["timestepper"]["tableau"] = "ImEx-BDF-3-3"
        inputfile["timestepper"]["eps_time"] = 1e-9
    return inputfile

In [6]:
v_n=[1,3,5]
v_multiple=[0.5,1,2,4]
for t in itertools.product( v_n, v_multiple) :
    inputfile = generate_standard_input( 
        t[0], t[1], "pointwise", "upwind-advection", "none")   
    try:
        print(t)
        m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)
        break

(1, 0.5)
(1, 1)
(1, 2)
(1, 4)
(3, 0.5)
(3, 1)
(3, 2)
(3, 4)
(5, 0.5)
(5, 1)
(5, 2)
(5, 4)


In [7]:
# Construct the Gaussian weights on our grids
def gauss_weights( js) :
    n = js["n"]
    Nx = js["Nx"]
    Ny = js["Ny"]
    lx = js["x"][1]-js["x"][0]
    ly = js["y"][1]-js["y"][0]
    hx = lx/Nx
    hy = ly/Ny
    (x,w) = np.polynomial.legendre.leggauss(js["n"])
    weights1dX = np.tile( w, js["Nx"])
    weights1dY = np.tile( w, js["Ny"])
    return np.reshape( np.kron( weights1dY, weights1dX)*hy/2.0*hx/2.0 , (js["n"]*js["Ny"], js["n"]*js["Nx"]))

In [8]:
last_out = list()
content = m.table()
for f in content:
    ncin = Dataset( m.select(f), 'r', format="NETCDF4")
    # time, xc, yc, vorticity, potential,
    # error(t), time_per_step(t), energy_1d(t), enstrophy_1d(t), vorticity_1d(t)
    inputfile = json.loads(ncin.inputfile)
    max_idx = ncin.variables["time"].shape[0]-1
    vo = ncin["vorticity"][:,:,:]
    weights = gauss_weights( f["grid"])
    tps = ncin["time_per_step"][:]
    ene = ncin["energy_1d"][:]
    ens = ncin["enstrophy_1d"][:]
    vor = ncin["vorticity_1d"][:]

    vor20 = 1./2.*np.sum( vo[0,:,:]**2*weights )
    vor21 = 1./2.*np.sum( vo[max_idx,:,:]**2*weights )
    vor40 = 1./4.*np.sum( vo[0,:,:]**4*weights )
    vor41 = 1./4.*np.sum( vo[max_idx,:,:]**4*weights )
    vor60 = 1./6.*np.sum( vo[0,:,:]**6*weights )
    vor61 = 1./6.*np.sum( vo[max_idx,:,:]**6*weights )
    e = dict()
    e["id"] = m.hashinput(f)
    #print( e["id"])
    e["n"] = inputfile["grid"]["n"]
    e["N"] = inputfile["grid"]["Nx"]
    e["error"] = float(ncin["error"][max_idx])
    e["vorticity2"] = ( vor21-vor20)/vor20 # same as enstrophy
    e["vorticity4"] = ( vor41-vor40)/vor40 
    e["vorticity6"] = ( vor61-vor60)/vor60 
    e["enstrophy"] = (ens[max_idx]-ens[0])/ens[0]
    e["energy"] = (ene[max_idx]-ene[0])/ene[0]
    e["vorticity"] = (vor[max_idx]-vor[0])

    last_out.append(e)


In [9]:
#define conversion function 
def orderToString(x): 
    if np.isnan(x) : return 'n/a'
    return'%.2f'% x

def errorToString(x):
    return '%.2e' % x

In [10]:
def generate_table( n):
    df = pd.json_normalize( last_out)
    df.set_index( "id", inplace=True)
    df = df.loc [ df["n"]==n]
    df = df.sort_values( by=['N']) # sort
    names = ["error", "vorticity2", "vorticity4", "vorticity6", "enstrophy", "energy", "vorticity"]
    for name in names:
        df.insert(df.columns.get_loc(name)+1,column="order."+name, value=df.loc[:,name])
        df["order."+name] = (np.log ( abs(df["order."+name]/ df["order."+name].shift(1)))/
            np.log( df["N"].shift(1)/df["N"]))
    df.set_index(['n','N'], inplace=True)
    df_red = df[[ "error", "order.error", "vorticity", "order.vorticity", "enstrophy",
                 "order.enstrophy", "energy", "order.energy"]]#
    headers = ["solution",
               "vorticity",
               "enstrophy",
               "energy"]
    df_red.columns=pd.MultiIndex.from_product([headers,["error", "order"] ])
    for header in headers :  
        df_red.loc[:, (header, "error")]=df_red.loc[:,(header, "error")].apply( errorToString)
        df_red.loc[:, (header, "order")]=df_red.loc[:,(header, "order")].apply( orderToString)
    return df_red

In [11]:
df1 = generate_table(1)
df3 = generate_table(3)
df5 = generate_table(5)

/home/matthias/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Convergence of upwind scheme

First, we test the upwind-advection scheme with pointwise multiplication

In [12]:
df1

solution        vorticity        enstrophy            energy       
          error order      error order      error  order      error  order
n N                                                                       
1 72   3.11e-01   n/a  -4.62e-06   n/a  -6.51e-03    n/a   7.32e-03    n/a
  144  1.74e-01  0.84  -5.87e-07  2.98  -2.67e-02  -2.03  -2.38e-02  -1.70
  288  9.15e-02  0.93  -7.70e-08  2.93  -2.12e-02   0.33  -2.03e-02   0.23
  576  4.68e-02  0.97  -7.49e-09  3.36  -1.28e-02   0.72  -1.25e-02   0.70

In [13]:
df3

solution       vorticity        enstrophy           energy      
          error order     error order      error order      error order
n N                                                                    
3 24   8.96e-03   n/a  1.72e-04   n/a  -2.52e-04   n/a  -1.83e-04   n/a
  48   1.63e-03  2.46  4.06e-06  5.41  -9.49e-06  4.73  -7.12e-06  4.68
  96   2.14e-04  2.93  8.39e-08  5.59  -4.13e-07  4.52  -2.59e-07  4.78
  192  2.53e-05  3.08  1.65e-09  5.67  -7.37e-08  2.49   5.76e-09  5.49

In [14]:
df5

solution       vorticity        enstrophy           energy       
         error order     error order      error order      error  order
n N                                                                    
5 12  1.12e-03   n/a  3.09e-07   n/a   3.52e-06   n/a   3.23e-06    n/a
  24  6.86e-05  4.03  8.42e-09  5.20  -1.69e-07  4.38  -3.52e-08   6.52
  48  2.70e-06  4.66  1.19e-11  9.46  -7.55e-08  1.16   9.19e-09   1.94
  96  1.02e-07  4.73  1.39e-14  9.74  -5.73e-08  0.40   1.10e-08  -0.26

In the last table we might see the effects of the accuracy of the third order time-stepper